In [3]:
# Import libraries
import requests
import pandas as pd

In [8]:
# Scrap the data from the Statistics Canada website
list_col = ['Year', 'Province', 'ISO-Code', 'Commodity', 'Type', 'Units', 'Value']
list_province = ['Newfoundland and Labrador', 'Prince Edward Island', 'Nova Scotia',
                 'New Brunswick', 'Quebec', 'Ontario', 'Manitoba', 'Saskatchewan', 'Alberta', 'British Columbia',
                 'Yukon', 'Northwest Territories', 'Nunavut']
list_iso_code = ['NL', 'PE', 'NS', 'NB', 'QC', 'ON', 'MB', 'SK', 'AB', 'BC', 'YT', 'NT', 'NU']

# Initialize an empty dataframe with the specified columns
dataframe = pd.DataFrame(columns=list_col)

# Iterate through the years from 2007 to 2018
for year in range(2007, 2019):
    # Construct the URL for the current year
    url = 'https://mmsd.nrcan-rncan.gc.ca/MIS/MISTable.aspx?FileT=01&Year=' + str(year)
    # Set the headers for the HTTP request
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53","X-Amzn-Trace-Id": "Root=1-628bf4ae-2e0289c6416939e92c947106"}
    # Make the GET request and retrieve the page content
    page = requests.get(url, headers=headers)
    # Parse the page content into a list of dataframes
    dfs = pd.read_html(page.text)
    
    # Iterate through the provinces
    for idx, prov in enumerate(list_province):
        
        # Make copies of the metals and nonmetals dataframes
        df_metals = dfs[0].copy()
        df_nonmetals = dfs[1].copy()
        
        # Process the metals data
        metals = df_metals
        # Drop the rows containing footnotes
        metals.drop(metals.tail(1).index, inplace=True)
        # Add the 'Type', 'Year', 'ISO-Code', and 'Province' columns
        metals['Type'] = 'metals'
        metals['Year'] = year
        metals['ISO-Code'] = list_iso_code[idx]
        metals['Province'] = prov
        # Reorder the columns to match the order in the 'list_col' variable
        metals = metals[['Year', 'Province', 'ISO-Code', 'Commodity', 'Type', 'Units', prov]]
        metals = metals.reindex(columns=['Year', 'Province', 'ISO-Code', 'Commodity', 'Type', 'Units', prov])
        metals.columns = list_col
        # Remove the footnotes from the 'Commodity' column
        metals.Commodity = metals.Commodity.apply(lambda x: x[: len(x) - 9] if 'Footnote' in x else x)

        # Process the nonmetals data
        nonmetals = df_nonmetals
        # Drop the rows containing footnotes
        nonmetals.drop(nonmetals.tail(1).index, inplace=True)
        # Add the 'Type', 'Year', 'ISO-Code', and 'Province' columns
        nonmetals['Type'] = 'nonmetals'
        nonmetals['Year'] = year
        nonmetals['ISO-Code'] = list_iso_code[idx]
        nonmetals['Province'] = prov
        # Reorder the columns to match the order in the 'list_col' variable
        nonmetals = nonmetals[['Year', 'Province', 'ISO-Code', 'Commodity', 'Type', 'Units', prov]]
        nonmetals = nonmetals.reindex(columns=['Year', 'Province', 'ISO-Code', 'Commodity', 'Type', 'Units', prov])
        nonmetals.columns = list_col
        # Remove the footnotes from the 'Commodity' column
        nonmetals.Commodity = nonmetals.Commodity.apply(lambda x: x[: len(x) - 9] if 'Footnote' in x else x)

        # Combine the metals and nonmetals data into a single dataframe for the current year
        yearly_dataframe = pd.concat([metals, nonmetals])
        # Append the yearly data to the overall dataframe
        dataframe = pd.concat([dataframe, yearly_dataframe])

In [10]:
# Clean the dataframe

# Replace certain strings in the 'Units' column with abbreviations
dataframe['Units'] = dataframe['Units'].replace(['$000', 'tonnes', 'kilotonnes', 'kilograms', '000 carats'], ['k$', 't', 'kt', 'kg', 'kct'])

# Replace the string 'x' with an empty string
dataframe = dataframe.replace(['x'], '')

# Save the modified data to a CSV file
dataframe.to_csv('Canadian_production_by_commodity.csv',index=False)

# Return the modified dataframe
dataframe

,Year,Province,ISO-Code,Commodity,Type,Units,Value
0,2007,Newfoundland and Labrador,NL,Antimony,metals,t,0
1,2007,Newfoundland and Labrador,NL,Antimony,metals,k$,0
2,2007,Newfoundland and Labrador,NL,Bismuth,metals,t,0
3,2007,Newfoundland and Labrador,NL,Bismuth,metals,k$,0
4,2007,Newfoundland and Labrador,NL,Cadmium,metals,t,0
...,...,...,...,...,...,...,...
50,2018,Nunavut,NU,Titanium dioxide,nonmetals,k$,0
51,2018,Nunavut,NU,Wollastonite,nonmetals,kt,0
52,2018,Nunavut,NU,Wollastonite,nonmetals,k$,0
53,2018,Nunavut,NU,Zeolite,nonmetals,kt,0
